# LLM Simplification Exploration

This notebook explores different models and prompting strategies for text simplification.

# 1. Global Setup
Run these cells once to initialize the environment, clients, and helper functions.

In [33]:
# Imports & Authentication
import os
try:
    from dotenv import load_dotenv
except ImportError:
    %pip install python-dotenv
    from dotenv import load_dotenv

from groq import Groq
from datasets import load_dataset

# Load keys
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# Initialize Client
if not api_key:
    print("❌ ERROR: GROQ_API_KEY not found in .env")
else:
    print("✅ Connected to Groq.")
    client = Groq(api_key=api_key)
    
    # List Available Models (So we don't guess)
    print("\n🔍 Available Models on Groq:")
    try:
        models = client.models.list()
        for m in models.data:
            print(f" - {m.id}")
    except Exception as e:
        print(f"⚠️ Could not list models: {e}")

✅ Connected to Groq.

🔍 Available Models on Groq:
 - openai/gpt-oss-safeguard-20b
 - llama-3.1-8b-instant
 - meta-llama/llama-guard-4-12b
 - meta-llama/llama-4-scout-17b-16e-instruct
 - openai/gpt-oss-20b
 - whisper-large-v3-turbo
 - openai/gpt-oss-120b
 - playai-tts-arabic
 - groq/compound-mini
 - qwen/qwen3-32b
 - playai-tts
 - moonshotai/kimi-k2-instruct
 - meta-llama/llama-prompt-guard-2-86m
 - groq/compound
 - meta-llama/llama-4-maverick-17b-128e-instruct
 - whisper-large-v3
 - moonshotai/kimi-k2-instruct-0905
 - meta-llama/llama-prompt-guard-2-22m
 - llama-3.3-70b-versatile
 - allam-2-7b


In [34]:
# Helper Function: Automated Evaluation
def evaluate_simplification(original, simplified, model_id="llama-3.1-8b-instant"):
    """
    Uses an LLM to evaluate the quality of the simplification.
    """
    if not simplified: return "No text to evaluate."
    
    try:
        prompt = f"""
        Compare the Original and Simplified texts.
        Rate the Simplified text on 1-10 for:
        1. Meaning Preservation (Did it lose info?)
        2. Simplicity (Is the language easier?)
        3. Structural Change (Did it change from paragraph to bullets?)

        Original: "{original[:500]}..."
        Simplified: "{simplified[:500]}..."
        
        Output format: 
        Meaning: [Score]/10
        Simplicity: [Score]/10
        Structure: [Comment]
        Analysis: [1 sentence reason]
        """
        
        completion = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )
        return completion.choices[0].message.content
        
    except Exception as e:
        return f"Error evaluating: {e}"

print("✅ Evaluation function defined.")

✅ Evaluation function defined.


In [35]:
# Load Test Data (WikiLingua)
try:
    print("⏳ Loading dataset slice (wiki_lingua)...")
    # Taking first example for consistency
    dataset = load_dataset("wiki_lingua", "english", split="train[:1]", trust_remote_code=True)
    test_text = dataset[0]['article']['document'][0]
    print(f"✅ Loaded Test Text: \"{test_text[:100]}...\"")
except Exception as e:
    print(f"⚠️ Failed to load dataset: {e}")
    # Fallback
    test_text = "Make sure that the area is a safe place, especially if you plan on walking home at night. It’s always a good idea to practice the buddy system."

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'wiki_lingua' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


⏳ Loading dataset slice (wiki_lingua)...
✅ Loaded Test Text: "make sure that the area is a safe place, especially if you plan on walking home at night.  It’s alwa..."


# 2. TEST A: Prompt Strategies (Llama 3.1 8B)
**Objective**: Compare how different prompts affect the style of simplification.

**Model**: `llama-3.1-8b-instant`

In [36]:
# Define Prompts for TEST A
test_a_prompts = {
    "Generic": "Simplify this text for a general audience.",
    "ELI5": "Explain this like I am 5 years old. Use short sentences and simple words.",
    "Lexical Only": "Keep the exact same sentence structure, but replace complex words with simpler synonyms. Do NOT use bullet points.",
    "Non-Native": "Rewrite this for someone learning English (Level A2). Avoid idioms and complex grammar."
}

MODEL_A = "llama-3.1-8b-instant"

print(f"🧪 Running Test A with Model: {MODEL_A}")
print(f"📝 INPUT TEXT:\n{test_text[:200]}...\n{'='*60}")

for name, system_prompt in test_a_prompts.items():
    print(f"\n▶️ STRATEGY: {name.upper()}")
    
    try:
        # 1. Run Simplification
        completion = client.chat.completions.create(
            model=MODEL_A,
            messages=[
                {"role": "system", "content": system_prompt}, 
                {"role": "user", "content": test_text}
            ],
            temperature=0.3
        )
        result = completion.choices[0].message.content
        
        # 2. Evaluate Result
        print(f"✨ Output:\n{result[:200]}... (truncated)")
        print("\n📊 Evaluation:")
        score = evaluate_simplification(test_text, result, model_id=MODEL_A)
        print(score)
        print("-"*60)
        
    except Exception as e:
        print(f"❌ Error: {e}")

🧪 Running Test A with Model: llama-3.1-8b-instant
📝 INPUT TEXT:
make sure that the area is a safe place, especially if you plan on walking home at night.  It’s always a good idea to practice the buddy system.  Have a friend meet up and walk with you. Research the ...

▶️ STRATEGY: GENERIC
✨ Output:
When going out, especially at night, it's essential to prioritize your safety. Here are some tips to help you get home safely:

1. **Buddy system**: Ask a friend to meet up and walk with you. This way... (truncated)

📊 Evaluation:
Meaning: 8/10
The simplified text lost some information about the importance of the area being a safe place, but it still conveys the main idea of prioritizing safety.

Simplicity: 9/10
The language in the simplified text is easier to understand, and the use of bullet points makes it more concise and readable.

Structure: The original text was in paragraph form, while the simplified text is in bullet points. This change in structure makes the text more scannable an

# 3. TEST B: Model Comparison (Llama 3.3 70B)
**Objective**: See if a larger/different model produces better quality simplifications using the same "Generic" prompt.

**Model**: `llama-3.3-70b-versatile` (or another from the list above)

In [ ]:
# NOTE: Check the output of '1. Global Setup' to see which models are available ID's.
MODEL_B = "llama-3.3-70b-versatile" 
prompt_b = "Simplify this text for a general audience."

print(f"🧪 Running Test B with Model: {MODEL_B}")

try:
    completion = client.chat.completions.create(
        model=MODEL_B,
        messages=[
            {"role": "system", "content": prompt_b}, 
            {"role": "user", "content": test_text}
        ],
        temperature=0.3
    )
    result_b = completion.choices[0].message.content
    
    print(f"✨ Output:\n{result_b[:300]}...")
    print("\n📊 Evaluation:")
    score = evaluate_simplification(test_text, result_b, model_id="llama-3.1-8b-instant") 
    print(score)
    
except Exception as e:
    print(f"❌ Error: {e}")

🧪 Running Test B with Model: llama-3.3-70b-versatile
✨ Output:
**Staying Safe While Traveling at Night**

When planning a night out, make sure the area is safe, especially if you'll be walking home. Here are some tips to help you stay safe:

1. **Walk with a friend**: Use the buddy system and walk with a friend, especially at night.
2. **Research public transpo...

📊 Evaluation:
Meaning: 8/10
The simplified text lost some information about the specific details of public transportation schedules and the potential cessation of services at night.

Simplicity: 9/10
The language used in the simplified text is easier to understand, and the use of bullet points makes it more concise and visually appealing.

Structure: The original text was in paragraph form, while the simplified text uses bullet points, which is a significant change in structure.

Analysis: The simplified text effectively conveys the main ideas of staying safe while traveling at night, but loses some of the specific details p